## **Этот ноутбук**: по вектора модели-исходника bge-m3 обучает модель fast-bgem3 повторят за учителем. Тут всё: от предобработки до теста.

In [ ]:

editor = 'kaggle'
base_link = '/content/' if editor=='colab' else '/kaggle/input/'

### объединяем данные

In [ ]:
# # # # 1. Монтируем Google Drive
# from google.colab import drive
# drive.mount('/content/drive')



# # 2. Копируем kaggle.json в нужную директорию
# !mkdir -p ~/.kaggle
# !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

# # 3. Устанавливаем пра3ва на файл
# !chmod 600 ~/.kaggle/kaggle.json
# # 4. Устанавливаем kaggle API (если не установлен)
# # !pip install -q kaggle

# # 5. Пример скачивания датасета
# dataset_name = 'kehhill/part-1-vectores'

# dataset_id = dataset_name.split('/')[-1]

# # # 5. Скачиваем датасет
# !kaggle datasets download -d {dataset_name}

# # # 6. Создаём папку и распаковываем тудк1а
# !mkdir -p {dataset_id}

# !unzip -q "{dataset_id}.zip" -d {dataset_id}

In [ ]:
# # # 5. Пример скачивания датасета
# dataset_name = 'kehhill/part-2-vectores'

# dataset_id = dataset_name.split('/')[-1]

# # # 5. Скачиваем датасет
# !kaggle datasets download -d {dataset_name}

# # # 6. Создаём папку и распаковываем тудк1а
# !mkdir -p {dataset_id}

# !unzip -q "{dataset_id}.zip" -d {dataset_id}

In [ ]:
# -*- coding: utf-8 -*-
import os, re, glob, numpy as np, pandas as pd
from typing import List, Tuple, Dict

PAIR_RE = re.compile(r".*?part(\d+)\.npz$")  # для сортировки по номеру

def collect_shard_pairs(dir_path: str) -> List[Tuple[str, str]]:
    """
    Находит пары (embeds_partNNN.npz, index_partNNN.csv) в папке и
    возвращает их отсортированными по NNN.
    """
    npz_files = sorted(glob.glob(os.path.join(dir_path, "*part*.npz")))
    pairs = []
    for npz_path in npz_files:
        m = PAIR_RE.match(npz_path)
        if not m:
            continue
        n = m.group(1)
        idx_path = os.path.join(dir_path, f"index_part{n}.csv")
        if os.path.exists(idx_path):
            pairs.append((npz_path, idx_path))
    # отсортируем по номеру
    pairs.sort(key=lambda p: int(PAIR_RE.match(p[0]).group(1)))
    return pairs

def collect_all_pairs(dirs: List[str]) -> List[Tuple[str, str]]:
    """
    Конкатенирует пары от нескольких подходов (папок) в том порядке,
    в каком папки указаны в списке dirs.
    """
    all_pairs = []
    for d in dirs:
        ps = collect_shard_pairs(d)
        if not ps:
            print(f"[warn] no shard pairs in: {d}")
        all_pairs.extend(ps)
    return all_pairs

def summarize_pairs(pairs: List[Tuple[str, str]]) -> pd.DataFrame:
    rows = []
    for npz_path, idx_path in pairs:
        with np.load(npz_path, mmap_mode="r") as z:
            n = z["layer_24"].shape[0]  # любая метка, строки совпадают
            d = z["layer_24"].shape[1]
        rows.append({"npz": os.path.basename(npz_path),
                     "index": os.path.basename(idx_path),
                     "rows": n, "dim": d})
    return pd.DataFrame(rows)


In [ ]:
p1, p2 = os.path.join(base_link, "part-1-vectores"), os.path.join(base_link, "part-2-vectores")

pairs = collect_all_pairs([p1, p2])
print(summarize_pairs(pairs))


### создаём тестовые данные

In [ ]:
import os, glob
import numpy as np
import pandas as pd

# где лежат пары файлов
DATA_DIR = p1[:]

# выбираем какой-то .npz (первый по сортировке)
npz_list = sorted(glob.glob(os.path.join(DATA_DIR, "embeds_part*.npz")))
assert len(npz_list) > 0, "Не нашёл embeds_part*.npz"
npz_path = npz_list[0]
print("Using:", npz_path)

# пытаемся найти соответствующий index_partXXX.csv
part_num = os.path.splitext(os.path.basename(npz_path))[0].split("part")[-1]
idx_path = os.path.join(DATA_DIR, f"index_part{part_num}.csv")
has_index = os.path.exists(idx_path)
if has_index:
    print("Index:", idx_path)
else:
    print("Index CSV not found — пропущу сохранение индекса")

# грузим .npz (меммапом можно, но тут маленький срез — можно и нормально)
z = np.load(npz_path)  # dtype fp16 внутри

# определяем сколько доступно и сколько откусить
N_total = z["layer_24"].shape[0]
N_take = min(20_000, N_total)
print(f"Total rows in shard: {N_total} -> taking: {N_take}")

# берём одинаковый срез по всем слоям
out = {
    "layer_13": z["layer_13"][:N_take],
    "layer_17": z["layer_17"][:N_take],
    "layer_24": z["layer_24"][:N_take],
}
# контроль dtypes
for k, v in out.items():
    if v.dtype != np.float16:
        out[k] = v.astype(np.float16)

# сохраняем тестовый набор
np.savez_compressed("test.npz", **out)
print("Saved test.npz:", {k: out[k].shape for k in out})

# (опционально) сохраним такие же первые строки индекса
if has_index:
    df = pd.read_csv(idx_path)
    df.head(N_take).to_csv("test_index.csv", index=False)
    print("Saved test_index.csv:", len(df.head(N_take)))


### генераторы

In [ ]:
import torch
from torch.utils.data import IterableDataset, DataLoader
from transformers import AutoTokenizer, AutoModel

class TeacherEmbStreamingDataset(IterableDataset):
    """
    Стриминг по парам (npz, csv). На каждом шарде:
      - читаем index_partNNN.csv (колонки: idx, text)
      - открываем embeds_partNNN.npz (ключи: layer_13, layer_17, layer_24) — fp16
    Выдаёт батчи токенов + teacher-эмбеддинги.
    """
    def __init__(self,
                 pairs: List[Tuple[str, str]],
                 tokenizer,
                 batch_size: int = 128,
                 max_len: int = 128,
                 text_col: str = "text",
                 device: str = "cpu"):
        self.pairs = pairs
        self.tok = tokenizer
        self.bs = batch_size
        self.max_len = max_len
        self.text_col = text_col
        self.device = device

    def _yield_batches_from_shard(self, npz_path: str, idx_path: str):
        df = pd.read_csv(idx_path)
        texts = df[self.text_col].astype(str).tolist()

        z = np.load(npz_path, mmap_mode="r")
        z13 = z["layer_13"]  # [N,D] float16
        z17 = z["layer_17"]
        z24 = z["layer_24"]
        N = len(texts)

        for start in range(0, N, self.bs):
            end = min(start + self.bs, N)
            batch_texts = texts[start:end]

            enc = self.tok(
                batch_texts,
                truncation=True, padding=True,
                max_length=self.max_len,
                return_tensors="pt"
            )

            # teacher эмбеддинги в torch (half), можно оставить на CPU — перенесёшь в train_step
            t13 = torch.tensor(z13[start:end], dtype=torch.float32)
            t17 = torch.tensor(z17[start:end], dtype=torch.float32)
            t24 = torch.tensor(z24[start:end], dtype=torch.float32)

            yield {
                "input_ids": enc["input_ids"],
                "attention_mask": enc["attention_mask"],
                # xlm-roberta — без token_type_ids
                "teacher": {13: t13, 17: t17, 24: t24},
                "texts": batch_texts,  # опционально, если нужно для отладки
            }

        z.close()

    def __iter__(self):
        # последовательный проход по всем парам
        for npz_path, idx_path in self.pairs:
            yield from self._yield_batches_from_shard(npz_path, idx_path)

def make_loader(pairs: List[Tuple[str, str]],
                tokenizer,
                batch_size: int = 128,
                max_len: int = 128,
                num_workers: int = 2,
                pin_memory: bool = True):
    ds = TeacherEmbStreamingDataset(
        pairs=pairs, tokenizer=tokenizer,
        batch_size=batch_size, max_len=max_len
    )
    # IterableDataset: shuffle тут не поддерживаем (нужно перешардировать заранее, если надо)
    return DataLoader(ds, batch_size=None, num_workers=num_workers, pin_memory=pin_memory)


In [ ]:

from torch.utils.data import IterableDataset, DataLoader
from transformers import AutoTokenizer, AutoModel

In [ ]:
# 2) токенайзер под student (xlm-roberta)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/queries/bge_fast_ten_lang/bge_fast_ten_lang", use_fast=True)
model = AutoModel.from_pretrained("/kaggle/input/queries/bge_fast_ten_lang/bge_fast_ten_lang")

# 3) лоадер
train_loader = make_loader(pairs, tokenizer, batch_size=128, max_len=42)

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

class TestDataset(Dataset):
    def __init__(self, npz_path: str, csv_path: str, tokenizer, max_len=128):
        self.data = np.load(npz_path)
        self.texts = pd.read_csv(csv_path)["text"].astype(str).tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        # вытаскиваем teacher эмбеддинги
        teacher = {
            13: torch.tensor(self.data["layer_13"][idx], dtype=torch.float16),
            17: torch.tensor(self.data["layer_17"][idx], dtype=torch.float16),
            24: torch.tensor(self.data["layer_24"][idx], dtype=torch.float16),
        }
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "teacher": teacher,
            "text": text
        }

# --- использование ---


test_ds = TestDataset("test.npz", "test_index.csv", tokenizer, max_len=42)
test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

# пример: берём один батч
for batch in test_loader:
    print(batch["input_ids"].shape)   # [B, L]
    print(batch["teacher"][24].shape)  # [B, D]
    break


### мультичекпоинт лосс

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def _mean_pool(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).type_as(last_hidden_state)  # [B,T,1]
    summed = (last_hidden_state * mask).sum(dim=1)                  # [B,D]
    denom = mask.sum(dim=1).clamp_min(1e-6)                         # [B,1]
    return summed / denom

class MultiCheckpointLoss(nn.Module):
    def __init__(self, student_to_teacher, weights=None):
        """
        student_to_teacher: dict {student_layer_idx: teacher_layer_idx}
        weights: dict {student_layer_idx: weight}, если None -> все = 1.0
        """
        super().__init__()
        self.student_to_teacher = student_to_teacher
        self.weights = weights or {Ls: 1.0 for Ls in student_to_teacher}
        self.cos = nn.CosineEmbeddingLoss()

    def forward(self, s_hidden, t_hidden, attention_mask):
        """
        s_hidden: dict {layer_idx: [B,T,D]}  (student hidden_states)
        t_hidden: dict {layer_idx: [B,D]}    (teacher pooled embeddings)
        attention_mask: [B,T]
        """
        total_loss = 0.0
        B = attention_mask.size(0)
        target = torch.ones(B, device=attention_mask.device)

        for Ls, Lt in self.student_to_teacher.items():
            s = _mean_pool(s_hidden[Ls], attention_mask)  # [B,D]
            t = t_hidden[Lt].detach()                     # [B,D]
            total_loss += self.weights.get(Ls, 1.0) * self.cos(s, t, target)

        return total_loss


### утилиты

In [ ]:
def build_layer_param_map(model):
    """
    Делает словарь {layer_idx: [имена параметров]} для encoder.layer.*,
    плюс "others" для всего остального.
    """
    layer_map = {}
    total = len(model.encoder.layer)

    for n, p in model.named_parameters():
        matched = False
        for i in range(total):
            if f"encoder.layer.{i}." in n:
                idx = i + 1  # 1-based индекс
                layer_map.setdefault(idx, []).append(n)
                matched = True
                break
        if not matched:
            layer_map.setdefault("others", []).append(n)

    return layer_map

layer_map = build_layer_param_map(model)

In [ ]:
def set_layer_lrs(model, assignments, base_opt=torch.optim.AdamW, layer_map=None, **opt_kwargs):
    """
    Если optimizer=None -> создаём новый AdamW с группами под assignments.
    Если optimizer передан -> пересоздаём его param_groups (без потери state).
    """
    assert layer_map is not None, "Нужен layer_map!"

    # проверяем корректность assignments
    all_parts = list(layer_map.keys())
    model_part_assign = [j for i in assignments for j in i[0]]
    missed_layers = [i for i in all_parts if i not in model_part_assign]
    bad_layers = [i for i in model_part_assign if i not in all_parts]
    assert missed_layers==[], f"Assignment should contain all layers that model has! You missed: {missed_layers}"
    assert bad_layers==[], f"You point layers that model hasn't: {bad_layers}"

    # строим группы
    param_groups = []
    for indices, val in assignments:
        for idx in indices:
            names = layer_map[idx]
            params = [p for n, p in model.named_parameters() if n in names]
            group = {"params": params, "name": f"layer_{idx}"}
            if val == "freeze":
                for p in params:
                    p.requires_grad = False
                group["lr"] = 0.0
                group["frozen"] = True
            else:
                for p in params:
                    p.requires_grad = True
                group["lr"] = float(val)
                group["frozen"] = False
            param_groups.append(group)


    optimizer = base_opt(param_groups, **opt_kwargs)

    return optimizer


In [ ]:
def scale_unfrozen_lrs(optimizer, factor):
    """
    action = 'increase' или 'decrease'
    factor = число, например 2.0
    """
    for g in optimizer.param_groups:
        if not g.get("frozen", False):
            g["lr"] *= factor


In [ ]:
def wurm_up_sheduler(optimizer, cur_step, wurm_up_steps=1000):
  if cur_step<=wurm_up_steps:
      scale_unfrozen_lrs(optimizer, cur_step/wurm_up_steps)

In [ ]:
model.to('cuda')
print()

In [ ]:
import torch
from torch.cuda.amp import autocast, GradScaler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_

# --- готовим модель, оптимайзер ---
device = "cuda"
model.to(device)

# оптимайзер
from torch.optim import AdamW
# [1,2,5,9,13,17,20,23,24]
# [1,2,3,4,5,6,7,8,9]

assign_1 = [(['others', 1,2], 'freeze'),([3,4,5,6], 1e-4),([7,8,9], 2e-4)]
assign_2 = [(['others', 1,2], 'freeze'),([3,4,5,6], 2e-5),([7,8,9], 4e-5)]
optimizer = set_layer_lrs(model, assign_1, base_opt=torch.optim.AdamW, layer_map=layer_map)

# grad scaler для FP16
scaler = GradScaler()

checnkpoint_layers = [4, 6, 9]


# лосс (по новым индексам)
student_to_teacher = {4: 13, 6: 17, 9: 24}  # пример маппинга

loss_fn = MultiCheckpointLoss(
    student_to_teacher=student_to_teacher,
    weights={4:0.2, 6:0.2, 9:1.0}  # кастомные веса
)

step_start_count = 0
phase = 1
val_freq = 2000 # in steps
# --- тренинг ---
num_epochs = 2
cnt_decrease_lr = 0
stop_training = False
base_patience = 1
patience = base_patience  # works only after phase 2
val_loss_dynamic = [1000]
count_running_savers_phase_1 = 0
step = 0
for epoch in range(num_epochs):

    running_loss = 0.0


    if stop_training: break
    for batch in train_loader:
        if stop_training: break

        step+=1

        ids = batch["input_ids"].to(device)
        attn = batch["attention_mask"].to(device)
        teacher_hiddens = {L: batch["teacher"][L].to(device) for L in batch["teacher"]}

        model.zero_grad(set_to_none=True)


        with autocast(enabled=True):
            s_out = model(
                input_ids=ids,
                attention_mask=attn,
                output_hidden_states=True,
                return_dict=True,
            )
            # забираем student эмбеддинги
            student_hiddens = {L: s_out.hidden_states[L] for L in checnkpoint_layers}

            # teacher храним/кастуем в float32 → лосс будет считаться стабильно
            teacher_hiddens = {L: teacher_hiddens[L].float() for L in teacher_hiddens}

            loss = loss_fn(student_hiddens, teacher_hiddens, attn)

        scaler.scale(loss).backward()

        # grad clipping (после unscale)
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        # шаг оптимизатора
        scaler.step(optimizer)
        scaler.update()



        running_loss += loss.item()


        if phase==2:
          wurm_up_sheduler(optimizer, step-step_start_count, wurm_up_steps=800)

        if step % 400 == 0:
            print(f"epoch {epoch} step {step} | loss {running_loss/400:.4f}")

            count_running_savers_phase_1+=1
            if phase==1 and (running_loss < 0.22 or epoch != 0):
                print('Loss reach phase 1 goal, it is turn for phase 2!')
                optimizer = set_layer_lrs(model, assign_2, base_opt=torch.optim.AdamW, layer_map=layer_map)
                step_start_count = step
                phase = 2
            if phase==1 and count_running_savers_phase_1==5:
                val_loss_dynamic.append(running_loss)
            running_loss = 0.0

        if phase==2 and (step-step_start_count)%val_freq==0:
          model.eval()
          val_loss = 0.0
          with torch.inference_mode():
              for batch in test_loader:
                  ids = batch["input_ids"].to(device)
                  attn = batch["attention_mask"].to(device)
                  teacher_hiddens = {L: batch["teacher"][L].to(device) for L in batch["teacher"]}

                  with autocast(enabled=True):
                      s_out = model(input_ids=ids, attention_mask=attn, output_hidden_states=True, return_dict=True)

                      student_hiddens = {L: s_out.hidden_states[L] for L in checnkpoint_layers}
                      loss = loss_fn(student_hiddens, teacher_hiddens, attn)
                  val_loss += loss.item()
          val_loss /= len(test_loader)

          val_loss_dynamic.append(val_loss)
          print(f"epoch {epoch} | val_loss={val_loss:.4f}")

          if min(val_loss_dynamic[:-1])*0.98<=val_loss:
              patience-=1
              cnt_decrease_lr += 1
              scale_unfrozen_lrs(optimizer, 1/2)
              if patience==0:
                print("Traing stopped on epoch {epoch+1}, step {step} due to no progress!")
                stop_training = True
              if cnt_decrease_lr==2:
                print("Traing stopped on epoch {epoch+1}, step {step} due to limit for decreasing lr!")
                stop_training = True
          if min(val_loss_dynamic[:-1])*0.98>val_loss:
            print('Save new best model with val loss ', val_loss)
            patience = base_patience
            save_small = 'bge_fast_ten_lang_pretrained'
            os.makedirs(save_small, exist_ok=True)
            model.save_pretrained(save_small)
          model.train()

    # валидация после эпохи





In [ ]:
print(val_loss_dynamic)